In [1]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from sklearn.utils import shuffle
from transformers import T5TokenizerFast, TFT5ForConditionalGeneration
from progressbar import progressbar as pb


2021-07-13 00:03:39.582081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-13 00:03:39.582309: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
data = pd.read_csv('new_dataset.csv',index_col=0)
data=data[['txt','PERSON','NORP','JOB','JOB1','GPE','ORG','TENSE']]
data['TENSE'] = np.where((data['TENSE'] == 'was'),'PAST','PRESENT')
data=shuffle(data)

n = 1000
df_train = data.iloc[:n,1:]
df_label = data.iloc[:n,0:1]


In [8]:
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5TokenizerFast.from_pretrained("t5-small")
optimizer = tf.keras.optimizers.Adam()

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [9]:
batch_size=32
num_of_batches=math.floor(len(df_train)/batch_size)

In [10]:
label=[df_label.values[i][0] for i in range (len(df_label))]
label_encoded=tokenizer.batch_encode_plus(label,padding=True,max_length=900,return_tensors='tf')["input_ids"]

inputbatch=['|'.join(df_train.iloc[i].dropna().values.tolist()) for i in range (len(df_train))]
input_encoded=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=900,return_tensors='tf')["input_ids"]

In [11]:
epoch=1
for k in range(epoch):

    running_loss=0
    
    
    for i in pb(range(num_of_batches)):
        input=input_encoded[i*batch_size:i*batch_size+batch_size]
        output=label_encoded[i*batch_size:i*batch_size+batch_size]
        
        #Entrainement sur le batch
        with tf.GradientTape() as tape:
            outputs = model(input_ids=input, labels=output)
            # Compute the loss value for this batch.
            loss_value = sum(outputs.loss)/len(outputs.loss)
            running_loss+=loss_value
        # Actualisation des poids
        gradients = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))



100% (31 of 31) |########################| Elapsed Time: 0:06:13 Time:  0:06:13


NameError: name 'train_loss' is not defined

In [15]:
val='|'.join(data.iloc[i,1:].dropna().values.tolist())
tok=tokenizer(val, return_tensors="tf").input_ids
print(val)
outputs = model.generate(tok)
outputs1=tokenizer.decode(outputs[0]).replace('<pad> ','').replace('</s>',"")
print(outputs1)
print(data.iloc[i,0])

Luca Crecco|Italian|footballer|Pescara|PRESENT
Luca Crecco is an Italian footballer who plays as a midfielder
Luca Crecco is an Italian footballer who plays as a left winger for Pescara
